In [1]:
import pandas as pd
from openai import AzureOpenAI
import json
import openai
import sys
import os
import pickle

In [2]:
sys.path.append(os.path.join(os.getcwd(), '..'))

In [3]:
from prompts import *

In [4]:
employment_ground_truth = pd.read_csv('../eval_employment_prompt.csv')
relation_ground_truth = pd.read_csv('../eval_relationship_prompt.csv')

In [5]:
employment_ground_truth.head()

,index,provider_type,patient_id,note_id,sentence_index,text,TRANSPORTATION_distance,TRANSPORTATION_resource,TRANSPORTATION_other,HOUSING_poor,...,EMPLOYMENT_student,SUPPORT_plus,SUPPORT_minus,PARENT,UNKNOWN,llm_text,llm_label_EMPLOYMENT,llm_label_UNKNOWN,EMPLOYMENT_nonadverse,EMPLOYMENT_adverse
0,0,Nursing,10954,401903,0,Ms. [**Known lastname 11692**] is a 52 year ol...,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,True,"{\n ""EMPLOYMENT"": ""UNKNOWN"",\n ""EMPLOYMENT_C...",UNKNOWN,True,False,False
1,1,Nursing,10954,401903,1,"During her admission, she developed obtundatio...",0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,True,"{\n ""EMPLOYMENT"": ""UNKNOWN"",\n ""EMPLOYMENT_C...",UNKNOWN,True,False,False
2,2,Nursing,10954,401903,2,She was extubated the next day after a thorace...,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,True,"{\n ""EMPLOYMENT"": ""UNKNOWN"",\n ""EMPLOYMENT_C...",UNKNOWN,True,False,False
3,3,Nursing,10954,401903,3,She was transferred to the medical service fro...,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,True,"{\n ""EMPLOYMENT"": ""UNKNOWN"",\n ""EMPLOYMENT_C...",UNKNOWN,True,False,False
4,4,Nursing,10954,401903,4,"Please see Vascular, SICU and Medicine admissi...",0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,True,"{\n ""EMPLOYMENT"": ""UNKNOWN"",\n ""EMPLOYMENT_C...",UNKNOWN,True,False,False


In [6]:
relation_ground_truth.head()

,index,provider_type,patient_id,note_id,sentence_index,text,TRANSPORTATION_distance,TRANSPORTATION_resource,TRANSPORTATION_other,HOUSING_poor,...,SUPPORT_plus,SUPPORT_minus,PARENT,UNKNOWN,llm_text,llm_label_marital_status,llm_label_RELATIONSHIP,llm_label_UNKNOWN,RELATIONSHIP_nonadverse,RELATIONSHIP_adverse
0,0,Nursing,10954,401903,0,Ms. [**Known lastname 11692**] is a 52 year ol...,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,True,"{\n ""marital_status"": ""UNKNOWN"",\n ""marital_...",UNKNOWN,UNKNOWN,True,False,False
1,1,Nursing,10954,401903,1,"During her admission, she developed obtundatio...",0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,True,"{\n ""marital_status"": ""UNKNOWN"",\n ""marital_...",UNKNOWN,UNKNOWN,True,False,False
2,2,Nursing,10954,401903,2,She was extubated the next day after a thorace...,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,True,"{\n ""marital_status"": ""UNKNOWN"",\n ""marital_...",UNKNOWN,UNKNOWN,True,False,False
3,3,Nursing,10954,401903,3,She was transferred to the medical service fro...,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,True,"{\n ""marital_status"": ""UNKNOWN"",\n ""marital_...",UNKNOWN,UNKNOWN,True,False,False
4,4,Nursing,10954,401903,4,"Please see Vascular, SICU and Medicine admissi...",0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,True,"{\n ""marital_status"": ""UNKNOWN"",\n ""marital_...",UNKNOWN,UNKNOWN,True,False,False


In [7]:
employment_ground_truth.columns

Index(['index', 'provider_type', 'patient_id', 'note_id', 'sentence_index',
       'text', 'TRANSPORTATION_distance', 'TRANSPORTATION_resource',
       'TRANSPORTATION_other', 'HOUSING_poor', 'HOUSING_undomiciled',
       'HOUSING_other', 'RELATIONSHIP_married', 'RELATIONSHIP_partnered',
       'RELATIONSHIP_divorced', 'RELATIONSHIP_widowed', 'RELATIONSHIP_single',
       'EMPLOYMENT_employed', 'EMPLOYMENT_underemployed',
       'EMPLOYMENT_unemployed', 'EMPLOYMENT_disability', 'EMPLOYMENT_retired',
       'EMPLOYMENT_student', 'SUPPORT_plus', 'SUPPORT_minus', 'PARENT',
       'UNKNOWN', 'llm_text', 'llm_label_EMPLOYMENT', 'llm_label_UNKNOWN',
       'EMPLOYMENT_nonadverse', 'EMPLOYMENT_adverse'],
      dtype='object')

In [8]:
relation_ground_truth.columns

Index(['index', 'provider_type', 'patient_id', 'note_id', 'sentence_index',
       'text', 'TRANSPORTATION_distance', 'TRANSPORTATION_resource',
       'TRANSPORTATION_other', 'HOUSING_poor', 'HOUSING_undomiciled',
       'HOUSING_other', 'RELATIONSHIP_married', 'RELATIONSHIP_partnered',
       'RELATIONSHIP_divorced', 'RELATIONSHIP_widowed', 'RELATIONSHIP_single',
       'EMPLOYMENT_employed', 'EMPLOYMENT_underemployed',
       'EMPLOYMENT_unemployed', 'EMPLOYMENT_disability', 'EMPLOYMENT_retired',
       'EMPLOYMENT_student', 'SUPPORT_plus', 'SUPPORT_minus', 'PARENT',
       'UNKNOWN', 'llm_text', 'llm_label_marital_status',
       'llm_label_RELATIONSHIP', 'llm_label_UNKNOWN',
       'RELATIONSHIP_nonadverse', 'RELATIONSHIP_adverse'],
      dtype='object')

In [9]:
relation_df = relation_ground_truth[['RELATIONSHIP_nonadverse', 'RELATIONSHIP_adverse']]

In [10]:
final_df = employment_ground_truth.join(relation_df, on='index', how='left')

In [11]:
final_df.head()

,index,provider_type,patient_id,note_id,sentence_index,text,TRANSPORTATION_distance,TRANSPORTATION_resource,TRANSPORTATION_other,HOUSING_poor,...,SUPPORT_minus,PARENT,UNKNOWN,llm_text,llm_label_EMPLOYMENT,llm_label_UNKNOWN,EMPLOYMENT_nonadverse,EMPLOYMENT_adverse,RELATIONSHIP_nonadverse,RELATIONSHIP_adverse
0,0,Nursing,10954,401903,0,Ms. [**Known lastname 11692**] is a 52 year ol...,0.0,0.0,0.0,0.0,...,0.0,0.0,True,"{\n ""EMPLOYMENT"": ""UNKNOWN"",\n ""EMPLOYMENT_C...",UNKNOWN,True,False,False,False,False
1,1,Nursing,10954,401903,1,"During her admission, she developed obtundatio...",0.0,0.0,0.0,0.0,...,0.0,0.0,True,"{\n ""EMPLOYMENT"": ""UNKNOWN"",\n ""EMPLOYMENT_C...",UNKNOWN,True,False,False,False,False
2,2,Nursing,10954,401903,2,She was extubated the next day after a thorace...,0.0,0.0,0.0,0.0,...,0.0,0.0,True,"{\n ""EMPLOYMENT"": ""UNKNOWN"",\n ""EMPLOYMENT_C...",UNKNOWN,True,False,False,False,False
3,3,Nursing,10954,401903,3,She was transferred to the medical service fro...,0.0,0.0,0.0,0.0,...,0.0,0.0,True,"{\n ""EMPLOYMENT"": ""UNKNOWN"",\n ""EMPLOYMENT_C...",UNKNOWN,True,False,False,False,False
4,4,Nursing,10954,401903,4,"Please see Vascular, SICU and Medicine admissi...",0.0,0.0,0.0,0.0,...,0.0,0.0,True,"{\n ""EMPLOYMENT"": ""UNKNOWN"",\n ""EMPLOYMENT_C...",UNKNOWN,True,False,False,False,False


In [12]:
len(final_df)

5321

In [22]:
with open('../azure_credentials.json', 'r') as file:
    azure_data = json.load(file)
    api_key = azure_data['API_KEY']
    api_version = azure_data['API_VERSION']
    azure_endpoint = azure_data['AZURE_ENDPOINT']
    azure_deployment_name = azure_data['AZURE_DEPLOYMENT_NAME']

client = AzureOpenAI(
    api_key=api_key,
    api_version=api_version,
    azure_endpoint = azure_endpoint
    )

deployment_name=azure_deployment_name #This will correspond to the custom name you chose for your deployment when you deployed a model. Use a gpt-35-turbo-instruct deployment.

# Defining a function to create the prompt from the instruction system message, the few-shot examples, and the current query
def create_prompt(system_message, user_message):    
    formatted_message = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message}
    ]
    
    return formatted_message

# This function sends the prompt to the GPT model
def send_message(message, model_name, max_response_tokens=500):
    response = client.chat.completions.create(
        model=model_name,
        messages=message,
        temperature=0,
        max_tokens=max_response_tokens,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )
    
    return response.choices[0].message.content.strip()

In [23]:
final_df['EMPLOYMENT_nonadverse'].value_counts()

EMPLOYMENT_nonadverse
False    5276
True       45
Name: count, dtype: int64

In [15]:
final_df.iloc[5320]

index                                                                    5327
provider_type                                                     Social Work
patient_id                                                              99469
note_id                                                                534350
sentence_index                                                             17
text                                   , LICSW #[**Numeric Identifier 2205**]
TRANSPORTATION_distance                                                   0.0
TRANSPORTATION_resource                                                   0.0
TRANSPORTATION_other                                                      0.0
HOUSING_poor                                                              0.0
HOUSING_undomiciled                                                       0.0
HOUSING_other                                                             0.0
RELATIONSHIP_married                                            

In [ ]:
index_list = []
llm_response_list = []
system_message = "You are an information extract tool that follows instructions very well and is specifically trained to extract social determinants of health elements from hospital generated free-text."

try:
    for index, row in final_df[0:500].iterrows():
        free_text = row['text']
        user_message = step1_query_optimized.format(free_text=free_text)
        openai_message = create_prompt(system_message, user_message)
        response = send_message(openai_message, deployment_name)
        
        index_list.append(index)
        llm_response_list.append(response)
        print(index)
        print(free_text)
        print(response)
        print()
except Exception as err:
    print("Something went wrong: ", err)

llm_0_500_step1 = pd.DataFrame({'index': index_list, 'llm_0_500_step1': llm_response_list})

with open('llm_0_500_step1.pkl', 'wb') as file:
    pickle.dump(llm_0_500_step1, file)

In [ ]:
index_list = []
llm_response_list = []
system_message = "You are an information extract tool that follows instructions very well and is specifically trained to extract social determinants of health elements from hospital generated free-text."

try:
    for index, row in final_df[500:1000].iterrows():
        free_text = row['text']
        user_message = step1_query_optimized.format(free_text=free_text)
        openai_message = create_prompt(system_message, user_message)
        response = send_message(openai_message, deployment_name)
        
        index_list.append(index)
        llm_response_list.append(response)
        print(index)
        print(free_text)
        print(response)
        print()
except Exception as err:
    print("Something went wrong: ", err)
    
llm_500_1000_step1 = pd.DataFrame({'index': index_list, 'llm_500_1000_step1': llm_response_list})

with open('llm_500_1000_step1.pkl', 'wb') as file:
    pickle.dump(llm_500_1000_step1, file)